In [9]:
from osgeo import gdal,ogr
import os
import shutil
import numpy as np
from datetime import datetime
import time

In [10]:
datapath ='E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Saint Martin'

savePath = "E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Radiometric_Correction"

if not os.path.exists(savePath):
        os.makedirs(savePath)
    

shapefile = "E:\\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\\Data File\\Shapefile\\New_Shapefile.shp"
        
        
FileList = {}
FileCount = 0
for root, dirs, files in os.walk(datapath):
    for file in files:
        filename = file[:26]
        if filename not in FileList:
            FileList[filename] = [root+'/'+file]
        else:
            FileList[filename].append(root+'/'+file)
        FileCount+=1
        

print(f"FileCount : {FileCount}")



FileCount : 207


In [11]:
# Print The image Count in each line
for i in FileList.keys():
    print(f"Count on {i} is {len(FileList[i])}")
    print("-"*10)

Count on 20230425_040443_31_24a3_3B is 3
----------
Count on 20230425_040445_47_24a3_3B is 3
----------
Count on 20230426_040518_97_24a4_3B is 3
----------
Count on 20230427_032740_97_241d_3B is 3
----------
Count on 20230427_032743_14_241d_3B is 3
----------
Count on 20230428_040948_46_2424_3B is 3
----------
Count on 20230428_041907_07_2426_3B is 3
----------
Count on 20230428_041909_10_2426_3B is 3
----------
Count on 20230428_041911_13_2426_3B is 3
----------
Count on 20230430_041121_66_2254_3B is 3
----------
Count on 20230430_041123_82_2254_3B is 3
----------
Count on 20230501_040624_83_2470_3B is 3
----------
Count on 20230511_031337_96_2445_3B is 3
----------
Count on 20230511_031340_24_2445_3B is 3
----------
Count on 20230511_033436_58_24ab_3B is 3
----------
Count on 20230511_033438_82_24ab_3B is 3
----------
Count on 20230517_041835_72_2414_3B is 3
----------
Count on 20230517_041837_75_2414_3B is 3
----------
Count on 20230519_033144_10_24b4_3B is 3
----------
Count on 202

In [12]:
# Extract Coefficient
from xml.dom import minidom
def Extract_Coeff(path):
    xml_path = path
    xmldoc = minidom.parse(xml_path)
    nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

    # XML parser refers to bands by numbers 1-4
    coeffs = {}
    for node in nodes:
        bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
        if bn in ['1', '2', '3', '4','5','6','7','8']:
            i = int(bn)
            value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
            coeffs[i] = float(value)
    return coeffs


In [13]:
#Extracting Band info
# Band 1 = Coastal Blue
# Band 2 = Blue
# Band 3 = Green I
# Band 4 = Green
# Band 5 = Yellow
# Band 6 = Red
# Band 7 = Red Edge
# Band 8 = Near-infrared
import rasterio
import numpy as np


def Extract_image_band(path):
    ds = rasterio.open(path)
    band = {}
    band['ds'] = ds
    band['coastal_blue'] = ds.read(1)
    band['blue'] = ds.read(2)
    band['green1'] = ds.read(3)
    band['green'] = ds.read(4)
    band['yellow'] = ds.read(5)
    band['red'] = ds.read(6)
    band['red_edge'] = ds.read(7)
    band['near_infrared'] = ds.read(8)
    
    return band


In [14]:
error = []

In [15]:
#Extract the savePath
monthList = {
    1: '1. Jan',
    2: '2. Feb',
    3: '3. Mar',
    4: '4. Apr',
    5: '5. May',
    6: '6. Jun',
    7: '7. July',
    8: '8. Aug',
    9: '9. Sep',
    10: '10. Oct',
    11: '11. Nov',
    12: '12. Dec'
}

def Extract_savePath(date):
    year = date.year
    month = date.month
    month = monthList[month]
    path = f'{savePath}/{year}/{month}'
    if not os.path.exists(path):
        os.makedirs(path)
    return path
        
    

In [16]:

for i in FileList.keys():
    flagM = 0
    flagI = 0
    flagU = 0
    band = {}
    coeff = {}
    Ipath = ""
    xmlpath = ""
    date = i.split('_')[0]
    date = datetime.strptime(date, "%Y%m%d")
    path = Extract_savePath(date)
    for file in FileList[i]:
        
        if file.endswith("udm2.tif"):
            shutil.copy(file, path)
            
            print(f"udm2 file copied for {i}")
            
            flagU = 1
            
        if file.endswith('metadata.xml'):
            xmlpath = file
            coeff = Extract_Coeff(xmlpath)
            print("Conversion coefficients: {}".format(coeff))
            flagM = 1
            print(len(coeff))
        if file.endswith("harmonized.tif"):
            name = file.split("/")[-1]
            print(f"Name : {name}")
            Ipath = file
            band = Extract_image_band(Ipath)
            print(f"Band Extracted from {Ipath}")
            flagI = 1
            
        if len(coeff) != 8:
            print(f"Conversion Coefficients are not available for {i}")
            flagM = 0
            flagI = 0
        if flagM == 1 and flagI == 1 and flagU == 1  and len(coeff) == 8:
            
            # Apply the conversion coefficient
            band['coastal_blue'] = band['coastal_blue']*coeff[1]
            band['blue'] = band['blue']*coeff[2]
            band['green1'] = band['green1']*coeff[3]
            band['green'] = band['green']*coeff[4]
            band['yellow'] = band['yellow']*coeff[5]
            band['red'] = band['red']*coeff[6]
            band['red_edge'] = band['red_edge']*coeff[7]
            band['near_infrared'] = band['near_infrared']*coeff[8]
            print(f"Calculations Done for {i}")
            
            # Save the image and Delete the previous one and metadata
            kwargs = band['ds'].meta
            kwargs.update(
                dtype=rasterio.uint16,
                count = 8)
            scale = 10000
            band['coastal_blue'] = (band['coastal_blue']*scale).astype(rasterio.uint16)
            band['blue'] = (band['blue']*scale).astype(rasterio.uint16)
            band['green1'] = (band['green1']*scale).astype(rasterio.uint16)
            band['green'] = (band['green']*scale).astype(rasterio.uint16)
            band['yellow'] = (band['yellow']*scale).astype(rasterio.uint16)
            band['red'] = (band['red']*scale).astype(rasterio.uint16)
            band['red_edge'] = (band['red_edge']*scale).astype(rasterio.uint16)
            band['near_infrared'] = (band['near_infrared']*scale).astype(rasterio.uint16)
            with rasterio.open(path + "\\" + name, 'w', **kwargs) as dst:
                dst.write(band['coastal_blue'], 1)
                dst.write(band['blue'], 2)
                dst.write(band['green1'], 3)
                dst.write(band['green'], 4) 
                dst.write(band['yellow'], 5)
                dst.write(band['red'], 6)
                dst.write(band['red_edge'], 7)
                dst.write(band['near_infrared'], 8)
            print(f"Image Saved for {i}")
            # Delete the previous image and metadata
            
            flagM = 0
            flagI = 0
            flagU = 0
            print(f"Conversion Done for {i}\n\n")
            
            dst.close()
            band['ds'].close()
            # time.sleep(2)
    for x in FileList[i]:
        try:
            os.remove(x)
            print(f"Deleted {x}")
        except:
            try:
                shutil.rmtree(x)
                print(f"Deleted {x}")
            except:
                error.append(x)
                print(f"Error in deleting {x}")
    print('-'*30)

Conversion coefficients: {1: 1.8574991045081444e-05, 2: 1.7939001877004983e-05, 3: 1.9172650100065854e-05, 4: 1.9456240842288822e-05, 5: 2.0910890874309942e-05, 6: 2.3418272006787264e-05, 7: 2.503558108448133e-05, 8: 3.706523633484699e-05}
8
Name : 20230425_040443_31_24a3_3B_AnalyticMS_SR_8b_harmonized.tif
Band Extracted from E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Saint Martin\2023\4. Apr/20230425_040443_31_24a3_3B_AnalyticMS_SR_8b_harmonized.tif
udm2 file copied for 20230425_040443_31_24a3_3B
Calculations Done for 20230425_040443_31_24a3_3B
Image Saved for 20230425_040443_31_24a3_3B
Conversion Done for 20230425_040443_31_24a3_3B


Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Saint Martin\2023\4. Apr/20230425_040443_31_24a3_3B_AnalyticMS_8b_metadata.xml
Deleted E:\LULC Change Detection Using Unsupervised Clustering of Saint Martin Bangladesh\Data File\Saint Martin\2023\4. Apr/202

In [17]:
# delete the error files
for x in error:
    try:
        os.remove(x)
        print(f"Deleted {x}")
    except:
        print(f"Error in deleting {x}")